In [5]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import root_scalar
from  rotation3sphere import align_to_north_pole

In [6]:
def fibonaccigrid_sampling(n):
    phi = (1 + np.sqrt(5)) / 2  
    
    # Initialize arrays for Cartesian coordinates
    points = np.zeros((n, 4))
    
    for k in range(n):
        #Compute angles using the index k and n points
        theta = np.arccos(1 - 2 * (k + 0.5) / n)  # Range [0, π]
        phi_k = 2 * np.pi * (k / phi**2)          # Range [0, 2π], scaled by golden ratio squared
        psi_k = 2 * np.pi * (k / phi**3)          # Additional rotation for 3rd angle, golden ratio cubed
        
        #Convert spherical coordinates to Cartesian coordinates in 4D
        points[k, 0] = np.cos(theta)
        points[k, 1] = np.sin(theta) * np.cos(phi_k)
        points[k, 2] = np.sin(theta) * np.sin(phi_k) * np.cos(psi_k)
        points[k, 3] = np.sin(theta) * np.sin(phi_k) * np.sin(psi_k)
    
    return points

In [7]:
def vertical_projection(v):
    """
    Compute the vertical projection of v at p=(1,0,0,0) in real coordinates.
    
    Parameters:
        v (np.ndarray): Tangent vector in R^4.

    Returns:
        np.ndarray: Vertical component of v.
    """
    if len(v) == 2:
        v = np.array([0, 0, v[0], v[1]])
        
    #it is already a normed vector
    vertical_direction = np.array((1, -1, 0, 0))  
    projection = np.dot(v, vertical_direction) * vertical_direction
    return projection

def horizontal_projection(v):
    """
    Project v into the horizontal space at p.
    
    Parameters:
        v (np.ndarray): Tangent vector in R^4.

    Returns:
        np.ndarray: Horizontal component of v.
    """
    vert_proj = vertical_projection(v)
    return v - vert_proj


In [8]:
def geodesic_error(params, q):
    """
    Compute the error for the sub-Riemannian geodesic with fixed s = 1 and 
    constrained v (v0 = 0, v1 = |vertical_projection(v)|).
    """
    p = np.array((1, 0, 0, 0))
    v2, v3 = params  
    v = np.array([0, 0, v2, v3])  
    norm_v = np.linalg.norm(v)

    if norm_v < 1e-10:
        return np.inf  # Prevent division by zero
    
    # Riemannian geodesic
    gamma_R = p * np.cos(norm_v) + (v / norm_v) * np.sin(norm_v)
    
    return np.linalg.norm(gamma_R - q)**2

In [9]:
def find_subriemannian_geodesic(q):
    """
    Find the initial velocity v such that the sub-Riemannian geodesic passes through q,
    with fixed s = 1 and constrained v.
    """
    p = np.array((1, 0, 0, 0))
    initial_v2_v3 = (1.0,1.0)
    v= np.array((0, 0, initial_v2_v3[0], initial_v2_v3[1]))

    """ constraint = {
        'type': 'eq',
        'fun': lambda v2_v3: np.linalg.norm(vertical_projection([0, 0, v2_v3[0], v2_v3[1]]))
    } """

    # Minimize the geodesic error
    result = minimize(
        geodesic_error,
        initial_v2_v3,
        args=(q,),
        #constraints=constraint,
        bounds=[(None, None), (None, None)],  
    )

    print(np.linalg.norm(np.array([0, 0, initial_v2_v3[0], initial_v2_v3[1]])))
    
    if result.success:
        v2, v3 = result.x
        v = np.array([0, 0, v2, v3])  
        return v
    else:
        raise ValueError("Optimization failed to find v. Reason: {}".format(result.message))

In [10]:
points = fibonaccigrid_sampling(10)
for p in points:
    v = find_subriemannian_geodesic(p)
    print("Initial point: ", (1,0,0,0))
    print("Target point: ", p)
    print("Initial velocity: ", v)   

1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [0.9        0.43588989 0.         0.        ]
Initial velocity:  [ 0.00000000e+00  0.00000000e+00 -4.54934387e-09 -4.54934387e-09]
1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [ 0.7        -0.52658671  0.04217387  0.48054948]
Initial velocity:  [0.         0.         0.05275613 0.60109198]
1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [ 0.5         0.0757129   0.84952161 -0.15026841]
Initial velocity:  [ 0.          0.          1.02954908 -0.18211917]
1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [ 0.3         0.58041368 -0.19653263 -0.73109157]
Initial velocity:  [ 0.          0.         -0.30983998 -1.15258397]
1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [ 0.1        -0.97977755 -0.1627927   0.05945163]
Initial velocity:  [ 0.          0.         -6.88582748  2.514696  ]
1.4142135623730951
Initial point:  (1, 0, 0, 0)
Target point:  [-0.1         0.839

In [11]:
def arccot(x):
    return np.arctan(1 / x) if x > 0 else np.pi / 2

In [12]:
def distance(q):
    """
    Compute the length of the sub-Riemannian geodesic from (1,0,0,0) to q.
    Parameters:
        q (np.ndarray): Target point in R^4.
    Returns:
        float: Length of the geodesic.
    """
    v = find_subriemannian_geodesic(q)
    v0,v1,v2,v3 = v
    u=v1/np.linalg.norm(v)
    rho = np.linalg.norm(v)
    z1 = np.exp(-1j*u*rho)*(np.cos(rho)+1j*u*np.sin(rho))
    z2 = (v2-1j*v3)/rho*np.exp(-1j*u*rho)*np.sin(rho)

    #case q is not on the vertical line or the horizontal sphere S2
    sigma1 = np.sign(np.cos(rho))
    sigma2 = np.sign(np.sin(rho))
    
    def target_function_case1(u):
        #choosing p=q=0
        term1 = arccot(np.sqrt(abs(z1)**2 - u**2) / (u * abs(z2)))
        term2 = u * arccot(np.sqrt(abs(z1)**2 - u**2) / abs(z2))
        theta1=arccot(np.sqrt(abs(z1)**2-u**2)/(u*abs(z2)))-u*arccot(np.sqrt(abs(z1)**2-u**2)/abs(z2))
        return term1 - term2 - theta1
    def target_function_case2(u):
        term1 = arccot(np.sqrt(abs(z1)**2 - u**2) / (u * abs(z2)))
        term2 = u * arccot(np.sqrt(abs(z1)**2 - u**2) / abs(z2))
        theta1=arccot(np.sqrt(abs(z1)**2-u**2)/(u*abs(z2)))-u*arccot(np.sqrt(abs(z1)**2-u**2)/abs(z2))+np.pi-u
        return term1 - term2 - theta1

    if sigma1>0 and sigma2>0:
        #distance should be positive and cannot be very big, as all points are in 
        #the sphere and it has maximal radius 1
        result = root_scalar(target_function_case1, method='brentq', bracket=(0,10))
        theta1=arccot(np.sqrt(abs(z1)**2-u**2)/(u*abs(z2)))-u*arccot(np.sqrt(abs(z1)**2-u**2)/abs(z2))
    if sigma1<0 and sigma2<0:
        #choosing p, q = 0
        result = root_scalar(target_function_case2, method='brentq', bracket=(0,10))
        theta1=arccot(np.sqrt(abs(z1)**2-u**2)/(u*abs(z2)))-u*arccot(np.sqrt(abs(z1)**2-u**2)/abs(z2))+np.pi-u
    if sigma1*sigma2==-1:
        #these two cases come down to the previous ones changing rho <- -rho
        rho=-rho
        result = root_scalar(target_function_case1, method='brentq', bracket=(0,10))
        theta1=arccot(np.sqrt(abs(z1)**2-u**2)/(u*abs(z2)))-u*arccot(np.sqrt(abs(z1)**2-u**2)/abs(z2))

    if result.converged:
        if abs(theta1)>0 and abs(theta1)<=np.pi/2*(1-abs(z1)):
            u0 = result.root
            d = np.sqrt(1-u0**2)*arccot(np.sqrt(abs(z1)**2 - u0**2) / abs(z2))
        else:
            print("abs(theta1) not within bounds of first case. Sample new points \
                  or implement difficult case")
    else:
        print("Root-finding did not converge.")

    return d

COMPUTE DISTANCE MATRIX

In [13]:
def a_matrix(points):
    A = np.zeros((len(points), len(points)))
    for i, p in enumerate(points):
        for j, q in enumerate(points):
            #punkte drehen
            p,q = align_to_north_pole(p,q)
            d = distance(q)
            A[i, j] = -0.5 * d**2
    return A

In [14]:
def b_matrix(points):
    A = a_matrix(points)
    H = np.identity(len(points)) - np.ones((len(points), len(points))) / len(points)
    #@ operator for matrix multiplication
    B = H@A@H
    return B